In [1]:
from ortools.linear_solver import pywraplp
import math

solver = pywraplp.Solver('SolveIntegerProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
objective = solver.Objective()
objective.SetMaximization()

In [2]:
# Never pick a game less likely than this
probability_threshold = 0.5

# Only look this far ahead
# On average contest goes to week 14.5
final_week_idx = 14

In [3]:
picks_made = [
    ['Chiefs',  'Bears', 'Colts'],
    ['Patriots', 'Titans', 'Steelers'],
    ['Ravens', 'Titans', 'Cardinals'],
#    ['49ers'], Eliminated week 1!
]

In [4]:
import csv
from collections import defaultdict

wins_by_team_and_week = defaultdict(lambda: [])

# NOTE: update these each week before running
living_picks = 3
current_week_idx = 2 # 0-based

for pick_idx in range(0, living_picks):
    pick_name = str(pick_idx + 1)
    
    wins_by_week = [[] for i in range(0, 17)]
    losses_by_week = [[] for i in range(0, 17)]
    wins_by_team = defaultdict(lambda: [])
    losses_by_team = defaultdict(lambda: [])
    
    with open('win_probabilities.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            week_idx, home_team, home_win_probability, away_team, away_win_probability = row
            week_idx = int(week_idx)            
            if week_idx > final_week_idx: # zero-based
                continue
            week_name = str(week_idx + 1)
            
            ## create variables for wins and losses
            var_name_format = 'W{:02d} E{:d} {:s} {:s} {:s} ({:f})'

            base_probability = float(home_win_probability)
            
            # regress p to coin flip based on current week (more uncertainty the farther we are from now)
            # this inflates win probablity if weeks_out is negative but those games are already decided so who cares
            regression_coeff = 0.8
            weeks_out = week_idx - current_week_idx
            p_ratio = base_probability / (1 - base_probability)
            regressed_p_ratio = (p_ratio - 1) * (regression_coeff ** weeks_out) + 1
            regressed_probability = regressed_p_ratio / (regressed_p_ratio + 1)
            p = regressed_probability
                        
            home_win = solver.IntVar(0, 1, var_name_format.format(week_idx + 1, pick_idx + 1, home_team, 'defeats', away_team, p))
            home_loss = solver.IntVar(0, 1, var_name_format.format(week_idx + 1, pick_idx + 1, home_team, 'loses to', away_team, 1.0 - p))
            away_win = solver.IntVar(0, 1, var_name_format.format(week_idx + 1, pick_idx + 1, away_team, 'defeats', home_team, 1.0 - p))
            away_loss = solver.IntVar(0, 1, var_name_format.format(week_idx + 1, pick_idx + 1, away_team, 'loses to', home_team, p))


            ## add win / loss constraints
            ## TODO I think there might be better syntax for this
            home_team_defeats_away_team = solver.Constraint(0,0)
            home_team_defeats_away_team.SetCoefficient(home_win, 1)
            home_team_defeats_away_team.SetCoefficient(away_loss, -1)
            away_team_defeats_home_team = solver.Constraint(0,0)
            away_team_defeats_home_team.SetCoefficient(home_loss, -1)
            away_team_defeats_home_team.SetCoefficient(away_win, 1)

            ## add value weights for win / loss picks
            ## this also implicitly prevents different entries from picking aginst themselves
            home_objective_score = -100 if p < probability_threshold else math.log(p) + 10
            away_objective_score = -100 if (1.0 - p) < probability_threshold else math.log(1.0 - p) + 10

            objective.SetCoefficient(home_win, home_objective_score)
            objective.SetCoefficient(away_win, away_objective_score)

            ## constrain win if already picked
            try:
                pick_already_made = picks_made[pick_idx][week_idx] # may be nil
                if pick_already_made == home_team:
                    solver.Constraint(1,1).SetCoefficient(home_win, 1)
                if pick_already_made == away_team:
                    solver.Constraint(1,1).SetCoefficient(away_win, 1)
            except IndexError:
                pass


            ## index variables for other constraints
            wins_by_week[week_idx].append(home_win)
            wins_by_week[week_idx].append(away_win)
            losses_by_week[week_idx].append(home_loss)
            losses_by_week[week_idx].append(away_loss)
            wins_by_team[home_team].append(home_win)
            wins_by_team[away_team].append(away_win)
            losses_by_team[home_team].append(home_loss)
            losses_by_team[away_team].append(away_loss)
            wins_by_team_and_week[home_team + '-' + week_name].append(home_win)
            wins_by_team_and_week[away_team + '-' + week_name].append(away_win)
                    
    ## Add additional constraints

    ## One win per week
    for week in wins_by_week:
        one_win_per_week = solver.Constraint(0, 1)
        for win in week:
            one_win_per_week.SetCoefficient(win, 1)

    ## One loss per week (TODO DRY out)
    for week in losses_by_week:
        one_loss_per_week = solver.Constraint(0, 1)
        for loss in week:
            one_loss_per_week.SetCoefficient(loss, 1)

    ## At most one win per team (whole season)
    for team_wins in wins_by_team.values():
        one_win_per_team = solver.Constraint(0, 1)
        for win in team_wins:
            one_win_per_team.SetCoefficient(win, 1)

    ## At most 3 losses per team (whole season)
    for team_losses in losses_by_team.values():
        three_losses_per_team = solver.Constraint(0, 3)
        for loss in team_losses:
            three_losses_per_team.SetCoefficient(loss, 1)
            
## Pick each team only once across all future entries
for key, team_and_week in wins_by_team_and_week.items():
    week_idx = int(key.split('-')[1]) - 1 # 0-based
    if week_idx >= current_week_idx:    
        pick_constraint = solver.Constraint(0, 1)
        for win in team_and_week:
            pick_constraint.SetCoefficient(win, 1)


In [5]:
## Solve it!
print("Starting solver...")
result_status = solver.Solve()
print("Done solving!")

Starting solver...
Done solving!


In [6]:
solver.Objective().Value()

428.3891842905495

In [7]:
result_status

0

In [8]:
wins = [item for sublist in wins_by_team_and_week.values() for item in sublist]

In [9]:
picks = list(filter(lambda win: win.solution_value() == 1.0, wins))
len(picks)

45

In [10]:
sorted(picks, key=lambda p: str(p))

[W01 E1 Chiefs defeats Texans (1.043478),
 W01 E2 Patriots defeats Dolphins (0.929624),
 W01 E3 Ravens defeats Browns (1.013699),
 W02 E1 Bears defeats Giants (0.824859),
 W02 E2 Titans defeats Jaguars (0.840909),
 W02 E3 Titans defeats Jaguars (0.840909),
 W03 E1 Colts defeats Jets (0.700000),
 W03 E2 Steelers defeats Texans (0.720000),
 W03 E3 Cardinals defeats Lions (0.700000),
 W04 E1 Rams defeats Giants (0.714286),
 W04 E2 Ravens defeats Washington (0.783105),
 W04 E3 Saints defeats Lions (0.664502),
 W05 E1 Cowboys defeats Giants (0.644675),
 W05 E2 49ers defeats Dolphins (0.635593),
 W05 E3 Rams defeats Washington (0.658341),
 W06 E1 Ravens defeats Eagles (0.651847),
 W06 E2 Bears defeats Panthers (0.597080),
 W06 E3 Colts defeats Bengals (0.599559),
 W07 E1 Bills defeats Jets (0.593628),
 W07 E2 Cowboys defeats Washington (0.600334),
 W07 E3 Chiefs defeats Broncos (0.645289),
 W08 E1 Titans defeats Bengals (0.623403),
 W08 E2 Colts defeats Lions (0.556514),
 W08 E3 Buccaneers d